In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import javabridge
import weka.core.jvm as jvm
from weka.classifiers import Classifier
from weka.core.converters import Loader
import weka.filters as filters
from weka.classifiers import Evaluation
from weka.filters import Filter
from weka.attribute_selection import ASSearch, ASEvaluation, AttributeSelection
import random
from weka.core.classes import Random

In [ ]:
import weka.core.jvm as jvm
jvm.start()

In [ ]:
# create a list of dataset filenames
dataset_filenames = ['data1 Sampled Scenarios.csv.arff', 'data2 Sampled Scenarios.csv.arff', 'data3 Sampled Scenarios.csv.arff', 'data4 Sampled Scenarios.csv.arff', 'data5 Sampled Scenarios.csv.arff',
                     'data6 Sampled Scenarios.csv.arff', 'data7 Sampled Scenarios.csv.arff', 'data8 Sampled Scenarios.csv.arff', 'data9 Sampled Scenarios.csv.arff', 'data10 Sampled Scenarios.csv.arff',
                     'data11 Sampled Scenarios.csv.arff', 'data12 Sampled Scenarios.csv.arff', 'data13 Sampled Scenarios.csv.arff', 'data14 Sampled Scenarios.csv.arff', 'data15 Sampled Scenarios.csv.arff']
dataset_filenames = ["Class/multiclass/" + filename for filename in dataset_filenames]

In [ ]:
# JRipper
loader = Loader(classname="weka.core.converters.ArffLoader")
for filename in dataset_filenames:
  data = loader.load_file(filename)
  data.class_is_last()
  # Split the selected dataset into training and testing sets
  train, test = data.train_test_split(80)
  # Build the classifier on the training data
  #To optimize the accuracy more increase -f values decrease -n value and increase -0 values
  cls = Classifier(classname="weka.classifiers.rules.JRip", options=["-F", "3", "-N", "2.0", "-O", "2"])
  cls.build_classifier(train)
  eval = Evaluation(train)
  random.seed(1)
  eval.crossvalidate_model(cls, test, 2, Random(1))
  print(eval.percent_correct)

In [ ]:
# Jripper + Adaboost
loader = Loader(classname="weka.core.converters.ArffLoader")
for filename in dataset_filenames:
  data = loader.load_file(filename)
  data.class_is_last()
  # split the selected dataset into training and testing sets
  train, test = data.train_test_split(80)

  # create a new Evaluation object for the selected attributes
  eval = Evaluation(train)

  # build the classifier on the training data
  #To optimize the accuracy more increase -f values decrease -n value and increase -0 values
  base_cls = Classifier(classname="weka.classifiers.rules.JRip", options=["-F", "3", "-N", "2.0", "-O", "2"])
  cls = Classifier(classname="weka.classifiers.meta.AdaBoostM1", options=["-P", "100", "-S", "1", "-I", "10", "-W", base_cls.classname, "--"])
  cls.build_classifier(train)

  # make predictions on the test data using the new Evaluation object
  random.seed(1)
  eval.crossvalidate_model(cls, test, 10, Random(1))
  print(eval.percent_correct)